# 03 智能搜索与推荐

本 Notebook 是智能商品搜索系统的最后一步：基于 LanceDB 中的向量数据，实现语义搜索、混合过滤、相似推荐和评论洞察。

**前置要求**：已运行 Notebook 02（LanceDB 中已有 `products` 和 `reviews` 两张表）

## 1. 连接数据库

加载 Notebook 02 写入的产品表和评论表。

In [ ]:
import os

import lancedb
import pandas as pd
from openai import OpenAI

# 连接 LanceDB
db = lancedb.connect("../lancedb_data")
products_table = db.open_table("products")
reviews_table = db.open_table("reviews")

print(f"数据库表: {db.table_names()}")
print(f"产品表: {products_table.count_rows()} 条")
print(f"评论表: {reviews_table.count_rows()} 条")

### 配置嵌入 API

搜索时需要将查询文本转为向量，复用 SiliconFlow API。

In [ ]:
assert os.environ.get("OPENAI_API_KEY"), "请设置环境变量 OPENAI_API_KEY"

client = OpenAI(
    api_key=os.environ["OPENAI_API_KEY"],
    base_url=os.environ.get("OPENAI_BASE_URL", "https://api.siliconflow.cn/v1"),
)

EMBEDDING_MODEL = "Qwen/Qwen3-Embedding-4B"


def get_query_vector(query: str) -> list[float]:
    """生成查询文本的嵌入向量"""
    response = client.embeddings.create(input=[query], model=EMBEDDING_MODEL)
    return response.data[0].embedding


print("API 配置完成")

## 2. 商品语义搜索

用自然语言描述需求，系统返回语义最匹配的商品。

这是向量搜索的核心价值：用户不需要知道精确的关键词，只需描述想要什么。

In [ ]:
def search_products(query: str, top_k: int = 5) -> pd.DataFrame:
    """语义搜索商品"""
    query_vec = get_query_vector(query)
    results = products_table.search(query_vec).limit(top_k).to_pandas()
    return results


def display_products(query: str, results: pd.DataFrame) -> None:
    """格式化展示搜索结果"""
    print(f"搜索: '{query}'\n")
    for i, (_, row) in enumerate(results.iterrows(), 1):
        print(f"  {i}. [{row['category']}/{row['subcategory']}] {row['brand']} - ¥{row['price']:.0f}")
        print(f"     {row['description'][:60]}")
        print(f"     评分: {row['rating']}  距离: {row['_distance']:.4f}")
    print()

In [ ]:
# 搜索手机
results = search_products("拍照好的手机")
display_products("拍照好的手机", results)

In [ ]:
# 搜索护肤品
results = search_products("适合敏感肌的护肤品")
display_products("适合敏感肌的护肤品", results)

In [ ]:
# 搜索户外装备
results = search_products("轻便防水的户外装备")
display_products("轻便防水的户外装备", results)

## 3. 评论语义搜索

在评论数据中搜索，找到与查询语义最相关的用户评价。

In [ ]:
def search_reviews(query: str, top_k: int = 5) -> pd.DataFrame:
    """语义搜索评论"""
    query_vec = get_query_vector(query)
    results = reviews_table.search(query_vec).limit(top_k).to_pandas()
    return results


def display_reviews(query: str, results: pd.DataFrame) -> None:
    """格式化展示评论搜索结果"""
    label_map = {1: "正面", 0: "负面"}
    print(f"搜索: '{query}'\n")
    for i, (_, row) in enumerate(results.iterrows(), 1):
        sentiment = label_map.get(row['label'], '未知')
        print(f"  {i}. [{row['cat']}] ({sentiment}) {row['review'][:70]}")
    print()

In [ ]:
# 搜索正面评价
results = search_reviews("质量好，非常满意")
display_reviews("质量好，非常满意", results)

In [ ]:
# 搜索负面评价
results = search_reviews("售后服务差，不推荐")
display_reviews("售后服务差，不推荐", results)

## 4. 混合搜索：语义 + 标量过滤

实际场景中，用户往往有多维度的筛选需求：既要语义匹配，又要满足价格、评分、类别等条件。

LanceDB 的 `.where()` 支持在向量搜索的基础上叠加 SQL 风格的过滤条件。

In [ ]:
def hybrid_search_products(
    query: str,
    category: str | None = None,
    max_price: float | None = None,
    min_rating: float | None = None,
    top_k: int = 5,
) -> pd.DataFrame:
    """混合搜索：语义 + 标量过滤"""
    query_vec = get_query_vector(query)
    search = products_table.search(query_vec).limit(top_k)

    filters: list[str] = []
    if category is not None:
        filters.append(f"category = '{category}'")
    if max_price is not None:
        filters.append(f"price <= {max_price}")
    if min_rating is not None:
        filters.append(f"rating >= {min_rating}")
    if filters:
        search = search.where(" AND ".join(filters))

    return search.to_pandas()

In [ ]:
# 1000 元以下的好手机
results = hybrid_search_products(
    "性价比高的手机",
    category="电子产品",
    max_price=1000,
    min_rating=4.0,
)
print("搜索: '性价比高的手机' (电子产品, ≤¥1000, 评分≥4.0)\n")
for i, (_, row) in enumerate(results.iterrows(), 1):
    print(f"  {i}. {row['brand']} {row['subcategory']} - ¥{row['price']:.0f} (评分 {row['rating']})")
    print(f"     {row['description'][:60]}")

In [ ]:
# 高评分的服装
results = hybrid_search_products(
    "保暖的冬季外套",
    category="服装",
    min_rating=4.5,
)
print("搜索: '保暖的冬季外套' (服装, 评分≥4.5)\n")
for i, (_, row) in enumerate(results.iterrows(), 1):
    print(f"  {i}. {row['brand']} {row['subcategory']} - ¥{row['price']:.0f} (评分 {row['rating']})")
    print(f"     {row['description'][:60]}")

In [ ]:
# 评论混合搜索：只看手机类的负面评论
query_vec = get_query_vector("屏幕质量问题")
results = (
    reviews_table
    .search(query_vec)
    .where("cat = '手机' AND label = 0")
    .limit(5)
    .to_pandas()
)
print("搜索: '屏幕质量问题' (手机类, 负面评论)\n")
for i, (_, row) in enumerate(results.iterrows(), 1):
    print(f"  {i}. [{row['cat']}] {row['review'][:70]}")

## 5. 相似商品推荐

给定一个商品，用它的向量在产品表中搜索最相似的商品。这是推荐系统的基础："看了这个商品的人还看了..."。

In [ ]:
def recommend_similar(
    product_id: str, top_k: int = 5
) -> tuple[pd.Series, pd.DataFrame]:
    """根据商品 ID 推荐相似商品"""
    # 获取目标商品的向量
    target = products_table.search().where(f"product_id = '{product_id}'").limit(1).to_pandas()
    if target.empty:
        raise ValueError(f"商品 {product_id} 不存在")

    target_row = target.iloc[0]
    target_vec = target_row["vector"]

    # 搜索相似商品（排除自身）
    results = (
        products_table
        .search(target_vec)
        .where(f"product_id != '{product_id}'")
        .limit(top_k)
        .to_pandas()
    )
    return target_row, results

In [ ]:
# 获取一个产品 ID 做演示
sample = products_table.search().limit(20).to_pandas()
# 选一个电子产品
electronics = sample[sample["category"] == "电子产品"]
if electronics.empty:
    electronics = sample
demo_product_id = electronics.iloc[0]["product_id"]

target, similar = recommend_similar(demo_product_id)

print(f"目标商品: [{target['category']}/{target['subcategory']}] {target['brand']} - ¥{target['price']:.0f}")
print(f"  {target['description'][:60]}")
print(f"\n相似商品推荐:")
for i, (_, row) in enumerate(similar.iterrows(), 1):
    print(f"  {i}. [{row['category']}/{row['subcategory']}] {row['brand']} - ¥{row['price']:.0f}")
    print(f"     {row['description'][:60]}")
    print(f"     距离: {row['_distance']:.4f}")

## 6. 评论洞察：跨维度组合搜索

同时搜索产品和评论，组合展示。例如：用户搜索 "续航好的手机"，系统同时返回匹配的商品和相关的用户评价。

这就是两个独立搜索维度的价值——产品描述告诉你 "商家说了什么"，评论告诉你 "用户怎么说"。

In [ ]:
def combined_search(query: str, top_k: int = 3) -> None:
    """组合搜索：同时搜索产品和评论"""
    query_vec = get_query_vector(query)

    # 搜索产品
    products = products_table.search(query_vec).limit(top_k).to_pandas()
    # 搜索评论
    reviews = reviews_table.search(query_vec).limit(top_k).to_pandas()

    label_map = {1: "正面", 0: "负面"}

    print(f"=== 搜索: '{query}' ===")
    print(f"\n--- 匹配商品 ---")
    for i, (_, row) in enumerate(products.iterrows(), 1):
        print(f"  {i}. [{row['subcategory']}] {row['brand']} - ¥{row['price']:.0f}")
        print(f"     {row['description'][:60]}")

    print(f"\n--- 相关评论 ---")
    for i, (_, row) in enumerate(reviews.iterrows(), 1):
        sentiment = label_map.get(row['label'], '未知')
        print(f"  {i}. [{row['cat']}] ({sentiment}) {row['review'][:70]}")
    print()

In [ ]:
combined_search("续航好的手机")

In [ ]:
combined_search("舒适的酒店住宿体验")

In [ ]:
combined_search("适合孩子看的书")

## 7. 回顾与总结

至此，我们完成了一个端到端的智能商品搜索系统。回顾整个系列：

| Demo | 技术 | 在本项目中的角色 |
|------|------|------------------|
| Demo 1 | Daft | 数据读取、清洗、转换（Notebook 01） |
| Demo 2 | Ray | 分布式加速嵌入生成（Notebook 02 扩展） |
| Demo 3 | LanceDB | 向量存储与语义搜索（Notebook 02-03） |
| Demo 4 | 综合 | 串联以上技术，构建完整 pipeline |

### 完整 pipeline 回顾

```
Raw Data (Demo1 + Demo3)
    --> Daft: Clean & Transform (Notebook 01)
    --> Daft embed_text: Generate Embeddings (Notebook 02)
    --> LanceDB: Store Vectors & Metadata (Notebook 02)
    --> Search & Recommend (Notebook 03)
        - Semantic Search
        - Hybrid Filtering
        - Similar Product Recommendation
        - Cross-dimension Insight
```

### 进一步探索

- **更大规模数据**：切换 Ray Runner 处理百万级数据
- **更好的搜索体验**：结合 LLM 做查询改写、结果摘要
- **实时更新**：增量写入新商品和评论，索引自动更新
- **多模态搜索**：用 Daft 的 `embed_image` 支持以图搜商品